# 通过钢铁侠变身快速理解Python的装饰器用法

## 1 一切都要从函数说起

我们都知道一个函数可以返回一些数据，然后这些数据可以被其他函数调用。函数里还可以有若干个参数，可以让函数根据不同的输入值进行不同的计算，然后得到新的结果。

于是，我们的故事就可以开始了。

现有第一个函数，tony，很简单，直接返回tony，然后你叫他一声。他就回应你：tony。

In [20]:
def tony():
    return 'tony'

In [3]:
tony()

'tony'

紧接着，我们来到第二个函数，他可以根据传入的一个男人man，然后说出谁是钢铁侠。

In [4]:
def iron_man(man):
    print(f'{man} is iron man!')

由于在Python里面任何东西都是个对象，函数也不例外，我们就直接把上面的`tony()`（注意括号啊！）直接传给`iron_man`，会得到什么呢？
如你所见，结果很简单，当做参数的函数，返回值传入了新的函数，然后组合出现了我们看到的结果。

In [5]:
iron_man(tony())

tony is iron man!


刚才只是简单的说一下谁是钢铁侠，下面他就要变身了。

既然函数可以当做参数，那么我们在定义复杂一点的函数。如下所示，把函数名func当做参数，然后嵌套写一个包装函数 wrapper，这个函数里面调用func，这时候是有括号的，当做他的变身。
同时这行代码的上下，分别添加语句，分别显示变身前和变身后。

这个函数的最后，返回的是内部包裹的函数名，其实就是个函数对象，但是，没有调用呢。

In [54]:
# tony要变身
def deco_iron(func):
    def wrapper():
        print('开始准备变身')
        man = func()
        print(f'{man} 正在变身!')        
        print('变身结束！')
    return wrapper

接下来，我们调用这个复杂函数，就可以完成钢铁侠的变身了。和刚才不同的是，注意我们只是传入函数名。因为这个函数要在刚才包裹他的函数内部运行。
我们可以给个新的变量new_tony，注意咯，这时候得到的是刚才复杂函数的返回值，也就是内部的函数对象。如果要调用，需要加括号的。

In [16]:
new_tony = deco_iron(tony)  # 注意！！没有括号，去变身里面执行

试一下就知道了。

In [18]:
new_tony()

开始准备变身
tony 正在变身!
变身结束！


如你所见，变身成功。下面问题来了：有没有可能直接运行tony就达到效果？

当然有，我们把刚才的返回值变量直接写作tony，运行看看：

In [21]:
tony = deco_iron(tony)
tony()

开始准备变身
tony 正在变身!
变身结束！


回顾一下，我们现在相当于还是调用名为tony的函数，但是，在没有改变原来函数的情况下，我们给他添加了整套的变身语句。如同钢铁盔甲增强了tony的身体素质，我们用了一个嵌套函数增强了刚才简简单单的tony函数，但其实从头到尾没有修改原来的tony函数。这就给代码增添了无限的可能。

但，这样调用来调用去的有点啰嗦，需要简化。

## 2 装饰器@魔法出现了

Python是一个以简洁著称的语言，刚才那种写完嵌套函数，调用一下，又返回同样的函数名，然后等着再调用的做法，直接可以用一个`@`符号搞定。
这时候的`@`相当于`tony = deco_iron(tony)`，然后直接加到原来的函数上头就ok了。如下所示：

In [55]:
@deco_iron
def tony():
    return 'tony'

调用一下这个函数，你会发现已经被装饰加强了。这就是装饰器了。

In [56]:
tony()

开始准备变身
tony 正在变身!
变身结束！


## 3 装饰器的变身

### 传递参数

函数是可以传递参数的，如果我们想让被装饰的函数传递参数可以这样做：

- 正常添加任何参数，比如我们这里加个msg
- 在装饰器函数的内部函数中，使用`*args, **kw`接收任何参数就ok了

为了刚方便查看测试效果，我们这里再用`func.__name__`格式化输出一下函数名。

In [57]:
# 传递参数
def deco_iron(func):
    print('deco_iron开始运行....')
    def wrapper(*args, **kw):
        print('开始变身，在执行{}函数前'.format(func.__name__))
        print(args, kw)
        func(*args, **kw)
        print('钢铁侠变身完成，结束{}函数'.format(func.__name__))
    return wrapper


下面，从新装饰一下tony函数，你会发现，定义之后就会自动运行上面装饰器函数的第一句话。其实这就是在Python内部，自动完成了`tony = deco_iron(tony)`，悄悄第一次运行了整个装饰器函数。接下来我们要调用一下下面的函数，才会运行外部函数的返回结果，从而进入的最内部的函数。

In [58]:
@deco_iron
def tony(msg):
    print(f'tony说：{msg}')


deco_iron开始运行....


In [30]:
tony('我来啦！')

开始变身，在执行tony函数前
('我来啦！',) {}
tony说：我来啦！
钢铁侠变身完成，结束tony函数


### 多重装饰器

单层的装饰器说完了。下面说一下多重装饰器。比如钢铁侠，有时候要去和更强大的敌人战斗，需要多穿一层盔甲。

于是，DE8UG新写个带new后缀的函数，同时在里面适当修改提示语句。

In [59]:
# 多重装饰器
def deco_iron_new(func):
    print('deco_iron_new开始运行....')
    def wrapper(*args, **kw):
        print('deco_iron_new 开始变身，在执行{}函数前'.format(func.__name__))
        print(args, kw)
        func(*args, **kw)
        print('deco_iron_new 钢铁侠变身完成，结束{}函数'.format(func.__name__))
    return wrapper

接下来就是使用了。只需要在要装饰的函数上，依次往上叠加@装饰函数名，就ok。这时候，定义语句被执行时候，你会发现同样执行了装饰器里面的第一句说明。

看一下执行顺序，是按照装饰顺序，从内到外执行的。

In [60]:
@deco_iron_new
@deco_iron
def tony(msg):
    print(f'tony说：{msg}')

deco_iron开始运行....
deco_iron_new开始运行....


但，我们需要注意的是，当实际执行tony函数时，会发生什么，先看结果再解释。

In [61]:
tony('我要超级变身')

deco_iron_new 开始变身，在执行wrapper函数前
('我要超级变身',) {}
开始变身，在执行tony函数前
('我要超级变身',) {}
tony说：我要超级变身
钢铁侠变身完成，结束tony函数
deco_iron_new 钢铁侠变身完成，结束wrapper函数


你会发现一个有意思的现象，实际执行的函数，是按照装饰的层次，从外到内执行了。

为什么呢？
仔细思考上文的装饰器语法含义，我们会发现刚才定义时，从内到外执行的多个装饰器函数，每个装饰器函数都返回一个函数对象。这时候最外层的返回函数对象当然就在最外面了。
当被装饰后的函数具体执行时，就像剥洋葱一样，从外到内，一层层的执行具体的语句。


### 发现函数名问题

相信眼尖的同学看出来了，刚才多层装饰器一次出现的说明语句，对于函数名的解析，似乎有点问题，最外层的没有正确说明要执行的tony函数。

这是为什么？

在函数执行的时候，会有自己的变量作用域，当多层嵌套的装饰器执行时，虽然运行不出错，但其实内部的作用域出现的混乱。比如我们看见的错误的显示了内嵌函数的名称。

这个怎么解决呢，Python自己带来的问题，当然可以自己解决。直接使用functools里的wraps装饰器，来修复这个作用域问题。用法就是在内嵌函数上装饰一下，记住原函数信息。

再次运行一下，你会发现函数名已经正常显示了。

In [63]:
from functools import wraps

# 多重装饰器
def deco_iron_new(func):
    print('deco_iron_new 开始运行....')
    @wraps(func)
    def wrapper(*args, **kw):
        print('deco_iron_new 开始变身，在执行{}函数前'.format(func.__name__))
        print(args, kw)
        func(*args, **kw)
        print('deco_iron_new 钢铁侠变身完成，结束{}函数'.format(func.__name__))
    return wrapper

# 传递参数
def deco_iron(func):
    print('deco_iron开始运行....')
    @wraps(func)
    def wrapper(*args, **kw):
        print('开始变身，在执行{}函数前'.format(func.__name__))
        print(args, kw)
        func(*args, **kw)
        print('钢铁侠变身完成，结束{}函数'.format(func.__name__))
    return wrapper

In [64]:
@deco_iron_new
@deco_iron
def tony(msg):
    print(f'tony说：{msg}')

deco_iron开始运行....
deco_iron_new 开始运行....


In [65]:
tony('再次变身')

deco_iron_new 开始变身，在执行tony函数前
('再次变身',) {}
开始变身，在执行tony函数前
('再次变身',) {}
tony说：再次变身
钢铁侠变身完成，结束tony函数
deco_iron_new 钢铁侠变身完成，结束tony函数


### 如果装饰器加参数呢

下面再来个复杂点的。既然装饰器本身还是函数，那么当然可以继续添加参数。

怎么加呢？
再嵌套一层函数，把参数传给它！

比如，钢铁侠要带着小辣椒一起飞，可以这么写：

In [49]:
# 如果装饰器加参数呢？
def deco_params(name):
    def deco_iron(func):
        print('deco_iron开始运行....')
        @wraps(func)
        def wrapper(*args, **kw):
            print('新参数需要处理：', name)
            print('开始变身，在执行{}函数前'.format(func.__name__))
            func(*args, **kw)
            print('钢铁侠变身完成，结束{}函数'.format(func.__name__))
        return wrapper
    return deco_iron


@deco_params('小辣椒')
def tony(msg):
    print(f'tony说：{msg}')

deco_iron开始运行....


In [50]:
tony('hi, 小辣椒')

新参数需要处理： 小辣椒
开始变身，在执行tony函数前
tony说：hi, 小辣椒
钢铁侠变身完成，结束tony函数


## 4 装饰器的用途

Python中，装饰器的用途很多。我们举一个web应用的例子。

我们都知道一个网站往往会有多个连接，根据不同的连接地址，进行路由转发，可以显示不同的业务内容。于是，我们可以这样定义一个类MyApp，表示网络应用。

里面放置三个函数，完成路由转发的功能：
- 初始化函数，放置一个字典
- 注册函数，用来当其他业务函数的装饰器。这是个嵌套函数，里面可以根据不同的路由页面名称，把函数名保存到字典
- 实际页面调用函数。

接下来使用时，可以很方便把不同页面的响应函数，注册到不同的路由下面。当访问不同页面时，根据响应的参数直接调用同一个方法，就能自动去转换到对应的页面内容了。

In [51]:
# 装饰器,各种用途
# author: De8uG


class MyApp():
    def __init__(self):
        self.func_map = {}
 
    def register(self, name):
        def func_wrapper(func):
            print('func_wrapper: ', name, func)
            self.func_map[name] = func
            return func
        return func_wrapper
 
    def call_method(self, name=None):
        func = self.func_map.get(name, None)
        if func is None:
            raise Exception("No function registered against - " + str(name))
        return name, func()


# 创建对象，用于添加方法进行调用
app = MyApp()

@app.register('/')
def main_page_func():
    return "This is the main page."


@app.register('/next_page')
def next_page_func():
    return "This is the next page."


print(app.call_method('/'))
print(app.call_method('/next_page'))

func_wrapper:  / <function main_page_func at 0x7ff4f109f9e0>
func_wrapper:  /next_page <function next_page_func at 0x7ff4f10e6440>
('/', 'This is the main page.')
('/next_page', 'This is the next page.')


ok，以上就是DE8UG给你带来的装饰器的相关知识了。你学会了吗？有任何问题，建议欢迎到「Python随身听」留言，感谢关注🤗。
